In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re
sum_model = "t5-small"

In [2]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

def removePattern(text, pattern):

    r = re.findall(pattern, text)

    for i in r:

        text = re.sub(i, '', text)

    return text

def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def sum_clean_text(text):
    if type(text)!=str:
        return " "
    text = remove_urls(text)
    text = cleanhtml(text)
    text = removePattern(text, "@[\w]*")  # remove handles
    text = removePattern(text, "&[\w]*")  # remove &amp
    return text

In [3]:
tokenizer = AutoTokenizer.from_pretrained(sum_model)
model = AutoModelForSeq2SeqLM.from_pretrained(sum_model)

In [4]:
prefix = "summarize: "

In [5]:
text ="""
The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.
"""

In [7]:
min_length=30
max_length=512
preprocess_text = sum_clean_text(text.strip().replace("\n",""))
t5_prepared_Text = "summarize: "+preprocess_text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt")
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=min_length,
                                    max_length=max_length,
                                    early_stopping=True)

In [8]:
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [9]:
output

'the president predicts some states will reopen this month. the u.s. has over 637,000 confirmed cases and over 30,826 deaths, the highest for any country in the world. "we want to get our country back," he says.'